In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2

from Network import Network
from MaxPooling import MaxPooling
from FullyConnected import FullyConnected
from Convolution import Convolution
from CrossEntropyLossFunction import CrossEntropyLossFunction
from Sigmoid import Sigmoid
from Softmax import Softmax
from Tanh import Tanh

In [8]:
images = []
labels = []


for folder in os.listdir('asl_dataset'):
    folder_dir = os.path.join('asl_dataset', folder)
    
    for image in os.listdir(folder_dir):
        img_path = os.path.join(folder_dir, image)
        
        data_sample = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        data_sample = cv2.resize(data_sample, (28,28))
        data_sample = data_sample / 255.0
        
        images.append(data_sample)
        labels.append(folder)        

print(labels)

['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9', '9',